In [1]:
import os
import tensorflow as tf
import numpy as np

# Utilities
import time
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import data
# batch_size = 500
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

batch_size = 128
from mlp.data_providers import MSD10GenreDataProvider, MSD25GenreDataProvider
train_data = MSD10GenreDataProvider('train', batch_size = batch_size, max_num_batches=5)
valid_data = MSD10GenreDataProvider('valid', batch_size = batch_size, max_num_batches=1)

In [3]:
# Parameters
learning_rate = 0.001
# training_iters = 100000
training_epochs = 50
display_step = 10

# Network Parameters
n_input = 25 # MNIST data input (img shape: 28*28)
n_steps = 120 # timesteps
n_hidden = 50 # dim(hidden layer)
n_layers = 5 # number of layers in cell
n_classes = 10 # MNIST total classes (0-9 digits)

In [4]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

tf.add_to_collection('vars', weights['out'])
tf.add_to_collection('vars', biases['out'])
saver = tf.train.Saver()

def LSTM(x, weights, biases): # Fully BPTT
    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, state_is_tuple=True)
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells=[lstm_cell] * n_layers, state_is_tuple=True)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)
    # [outputs' size is (?, n_steps, n_hidden), we want the last element, with index 120-1 for MSD
    # size of last_output is then (?, n_hidden)
    
#     print 'state_size = ', lstm_cell.state_size
    
    last_output = outputs[:, outputs.get_shape()[1] - 1, :]
    # Linear activation, using rnn inner loop last output
    return tf.matmul(last_output, weights['out']) + biases['out']

In [5]:
pred = LSTM(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [6]:
# Performance pickles
err_tr = np.zeros(training_epochs)
acc_tr = np.zeros(training_epochs)
err_val = np.zeros(training_epochs)
acc_val = np.zeros(training_epochs)

In [7]:
with tf.Session() as sess:
    print("Optimization Starts! Dim(hidden)={0:d}".format(n_hidden))
    sess.run(init) #initial run
        
    for e in range(training_epochs):                              
        if e == 0:
            start = time.time()
            num_batches_tr = 0
            num_batches_val = 0

        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:
            if e == 0:
                num_batches_tr += 1
            input_batch = input_batch.reshape((batch_size,n_steps,n_input))

            _, batch_error, batch_acc = sess.run(
                [optimizer, cost, accuracy], 
                feed_dict={x: input_batch, y: target_batch})
            running_error += batch_error
            running_accuracy += batch_acc
#             if num_batches_tr % 100 == 0:
#                 print('{0:02d}th batch: err(train)={1:.2f} acc(train)={2:.2f}'
#                       .format(num_batches_tr,running_error/num_batches_tr,running_accuracy/num_batches_tr))                      
        running_error /= num_batches_tr
        err_tr[e] = running_error
        running_accuracy /= num_batches_tr
        acc_tr[e] = running_accuracy

        if ((e+1) % display_step == 0) or (e == 0):
            end = time.time()
            print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f} ,,, taking {3:.1f}(sec)'
                  .format(e + 1, running_error, running_accuracy, end-start))

            start = time.time()            

            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                if e == 0:
                    num_batches_val += 1
                input_batch = input_batch.reshape((batch_size,n_steps,n_input))

                batch_error, batch_acc = sess.run(
                    [cost, accuracy], 
                    feed_dict={x: input_batch, y: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= num_batches_val
            err_val[e] = valid_error
            valid_accuracy /= num_batches_val
            acc_val[e] = valid_accuracy

            end = time.time()
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f} ,,, taking {2:.1f}(sec) '
                   .format(valid_error, valid_accuracy, end-start))
            
            start = time.time()

    sname = 'params_lstm_h' + str(n_hidden) +'_ly' + str(n_layers)
    saver.save(sess, sname)

    lstm_prf = {"err_tr": err_tr, "err_val": err_val, "acc_tr": acc_tr, "acc_val": acc_val}
    fname = "lstm_performances_h" + str(n_hidden) +'_ly' + str(n_layers) + ".p"
    pickle.dump(lstm_prf, open(fname,"wb"))

    print("Optimization Finished!")

Optimization Starts! Dim(hidden)=50
End of epoch 01: err(train)=1.91 acc(train)=0.32 ,,, taking 17.1(sec)
                 err(valid)=1.72 acc(valid)=0.38 ,,, taking 1.1(sec) 


KeyboardInterrupt: 

# Reference: 
(n_hidden, time(sec)_tr) per epoch: (10,17.1) (30,30.5) (50,49.5) (100,77.9) (200,171.0)

# Restore session

In [20]:
with tf.Session() as sess:
    print("<Restored> Optimization Starts! Dim(hidden)={0:d}".format(n_hidden))
    new_saver = tf.train.import_meta_graph('params_lstm_h5.meta')
    new_saver.restore(sess_saved, tf.train.latest_checkpoint('./'))
    
    vars_saved = tf.get_collection('vars')
    
    sess.run(init)    
    for e in range(training_epochs):                      
        if e == 0:
            start = time.time()
            num_batches_tr = 0
            num_batches_val = 0

        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:
            if e == 0:
                num_batches_tr += 1
            input_batch = input_batch.reshape((batch_size,n_steps,n_input))

            _, batch_error, batch_acc = sess.run(
                [optimizer, cost, accuracy], 
                feed_dict={x: input_batch, y: target_batch})
            running_error += batch_error
            running_accuracy += batch_acc
#             if num_batches_tr % 100 == 0:
#                 print('{0:02d}th batch: err(train)={1:.2f} acc(train)={2:.2f}'
#                       .format(num_batches_tr,running_error/num_batches_tr,running_accuracy/num_batches_tr))                      
        running_error /= num_batches_tr
        err_tr[e] = running_error
        running_accuracy /= num_batches_tr
        acc_tr[e] = running_accuracy

        if ((e+1) % display_step == 0) or (e == 0):
            end = time.time()
            print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f} ,,, taking {3:.1f}(sec)'
                  .format(e + 1, running_error, running_accuracy, end-start))

            start = time.time()            

            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                if e == 0:
                    num_batches_val += 1
                input_batch = input_batch.reshape((batch_size,n_steps,n_input))

                batch_error, batch_acc = sess.run(
                    [cost, accuracy], 
                    feed_dict={x: input_batch, y: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= num_batches_val
            err_val[e] = valid_error
            valid_accuracy /= num_batches_val
            acc_val[e] = valid_accuracy

            end = time.time()
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f} ,,, taking {2:.1f}(sec) '
                   .format(valid_error, valid_accuracy, end-start))
            
            start = time.time()

    sname = 'params_lstm_h' + str(n_hidden)
    saver.save(sess, sname)

    lstm_prf = {"err_tr": err_tr, "err_val": err_val, "acc_tr": acc_tr, "acc_val": acc_val}
    fname = "lstm_performances_h" + str(n_hidden) + ".p"
    pickle.dump(lstm_prf, open(fname,"wb"))

    print("Optimization Finished!")

<Restored> Optimization Starts! Dim(hidden)=5
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
Optimization Finished!


# ===== Worked for 1 epoch then Iter stopped ====

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
#         batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x, batch_y = train_data.next()
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

Iter 1000, Minibatch Loss= 2.541809, Training Accuracy= 0.32000
Iter 2000, Minibatch Loss= 2.333999, Training Accuracy= 0.27000
Iter 3000, Minibatch Loss= 1.916845, Training Accuracy= 0.37000
Iter 4000, Minibatch Loss= 1.882300, Training Accuracy= 0.37000
Iter 5000, Minibatch Loss= 1.661882, Training Accuracy= 0.45000
Iter 6000, Minibatch Loss= 1.590190, Training Accuracy= 0.44000
Iter 7000, Minibatch Loss= 1.780682, Training Accuracy= 0.36000
Iter 8000, Minibatch Loss= 1.716225, Training Accuracy= 0.43000
Iter 9000, Minibatch Loss= 1.640091, Training Accuracy= 0.40000
Iter 10000, Minibatch Loss= 1.656243, Training Accuracy= 0.37000
Iter 11000, Minibatch Loss= 1.606353, Training Accuracy= 0.46000
Iter 12000, Minibatch Loss= 1.530744, Training Accuracy= 0.48000
Iter 13000, Minibatch Loss= 1.729684, Training Accuracy= 0.37000
Iter 14000, Minibatch Loss= 1.772618, Training Accuracy= 0.42000
Iter 15000, Minibatch Loss= 1.725407, Training Accuracy= 0.40000
Iter 16000, Minibatch Loss= 1.6336

StopIteration: 

In [2]:
# Calculate accuracy for 128 mnist test images
# test_len = 100
# test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
# test_label = mnist.test.labels[:test_len]
# print("Testing Accuracy:", \
#     sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 1000, Minibatch Loss= 1.858730, Training Accuracy= 0.39000
Iter 2000, Minibatch Loss= 1.406672, Training Accuracy= 0.50000
Iter 3000, Minibatch Loss= 1.266393, Training Accuracy= 0.53000
Iter 4000, Minibatch Loss= 0.981411, Training Accuracy= 0.72000
Iter 5000, Minibatch Loss= 0.846963, Training Accuracy= 0.75000
Iter 6000, Minibatch Loss= 0.874808, Training Accuracy= 0.71000
Iter 7000, Minibatch Loss= 0.574552, Training Accuracy= 0.83000
Iter 8000, Minibatch Loss= 0.729932, Training Accuracy= 0.74000
Iter 9000, Minibatch Loss= 0.985042, Training Accuracy= 0.72000
Iter 10000, Minibatch Loss= 0.505130, Training Accuracy= 0.85000
Iter 11000, Minibatch Loss= 0.529583, Training Accuracy= 0.85000
Iter 12000, Minibatch Loss= 0.463187, Training Accuracy= 0.84000
Iter 13000, Minibatch Loss= 0.34425

KeyboardInterrupt: 

### MNIST works in below

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def RNN(x, weights, biases):
    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)
    # [outputs' size is (?, 28, 128), we want the last element, with index 27
    # size of last_output is then (?, 128)
    last_output = outputs[:, outputs.get_shape()[1] - 1, :]
    # Linear activation, using rnn inner loop last output
    return tf.matmul(last_output, weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 1.745764, Training Accuracy= 0.37500
Iter 2560, Minibatch Loss= 1.635502, Training Accuracy= 0.48438
Iter 3840, Minibatch Loss= 1.253110, Training Accuracy= 0.57812
Iter 5120, Minibatch Loss= 0.925527, Training Accuracy= 0.64844
Iter 6400, Minibatch Loss= 0.760226, Training Accuracy= 0.71094
Iter 7680, Minibatch Loss= 1.037197, Training Accuracy= 0.64062
Iter 8960, Minibatch Loss= 0.818278, Training Accuracy= 0.75000
Iter 10240, Minibatch Loss= 0.589991, Training Accuracy= 0.83594
Iter 11520, Minibatch Loss= 0.358511, Training Accuracy= 0.92188
Iter 12800, Minibatch Loss= 0.640873, Training Accuracy= 0.78125
Iter 14080, Minibatch Loss= 0.490448, Training Accuracy= 0.82812
Iter 15360, Minibatch Loss= 0.262663, Training Accuracy= 0.96094
Iter 16640, Minibatch Loss= 0.388

###################################

In [49]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [55]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [batch_size, n_steps, n_input])
y = tf.placeholder("float", [batch_size, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [75]:
def LSTM(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
            
    # Get lstm cell output
    outputs,states = tf.nn.dynamic_rnn(lstm_cell, x, n_steps)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [76]:
pred = LSTM(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

28


ValueError: Dimension must be 2 but is 3 for 'transpose_40' (op: 'Transpose') with input shapes: [128,28], [3].

In [ ]:

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})